In [128]:
import requests
import warnings
from bs4 import BeautifulSoup
import csv
from multiprocessing.pool import ThreadPool

In [129]:
warnings.filterwarnings("ignore")

In [130]:
def extract_article_data(url):
    try:
        headers = {'Content-Type': 'text/html'}
        getpage = requests.get(url, headers=headers, verify=False)
        soup = BeautifulSoup(getpage.text, 'html.parser')
        headline = soup.find("div", {"class": "article-header"}).h1.text.strip()
        content = soup.find("div", {"class": "description"}).text.strip()
        return headline, content
    except Exception as e:
        print(e, url)
        with open("recover.txt", "a+") as f:
            f.write(url + '\n')
        return None, None

In [131]:
def process_urls(urls):
    results = []
    with ThreadPool(processes=10) as pool:
        for result in pool.imap_unordered(extract_article_data, urls):
            if result[0] is not None:
                results.append(result)
    return results


In [132]:
if __name__ == "__main__":
    with open('ForExtractionOneYear7Pradesh2.csv', 'a+', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Headline", "Content"])
        with open("ForExtractionOneYear7Pradesh2.txt", 'r') as f:
            urls = f.read().splitlines()
            extracted_data = process_urls(urls)
            for headline, content in extracted_data:
                writer.writerow([headline, content])
